## Test notebook for the MegaDetector Flask API

### Imports and environment

In [ ]:
import requests
import os
import json
import io

from requests_toolbelt import MultipartEncoder
from requests_toolbelt.multipart import decoder
from PIL import Image

sample_input_dir = os.path.expanduser('~/git/CameraTraps/test_images/test_images')

ip_address = 'localhost'
port = '5050'
detect_endpoint = 'http://{}:{}/v1/camera-trap/sync/detect'.format(ip_address, port)
print(detect_endpoint)

max_images_per_call = 8

### Select images to submit

In [ ]:
filenames = os.listdir(sample_input_dir)
# filenames = [s for s in filenames if (s.lower().endswith('.png') or s.lower().endswith('.jpg'))]
filenames = [os.path.join(sample_input_dir,s) for s in filenames]
    
print('Found {} image(s):'.format(len(filenames)))
for fn in filenames:
    print(fn)

### Submit images

In [ ]:
params = {
    'min_confidence': 0.15,
    'min_rendering_confidence': 0.2,
    'render': True,
    'key': None
}

def clean_filename(s):
    s = s.replace('/','_').replace('\\','_').replace(':','_')
    return s
    
file_handles = {}

if len(filenames) > max_images_per_call:
    import random
    filenames_to_submit = random.sample(filenames,max_images_per_call)
else:
    filenames_to_submit = filenames
    
for fn in filenames_to_submit:
    file_handles[fn] = (clean_filename(fn), open(fn, 'rb'), 'image/jpeg')

m = MultipartEncoder(fields=file_handles)
print(m.content_type)

r = requests.post(detect_endpoint, 
                  params=params,
                  data=m,
                  headers={'Content-Type': m.content_type})

print('Status: {}'.format(r.status_code))

if not r.ok:
    print('Error: {}\n{}'.format(r.reason,r.text))
    results = None
else:
    results = decoder.MultipartDecoder.from_response(r)
    
print('Elapsed time: {}'.format(r.elapsed.total_seconds()))

for f in file_handles.values():
    f[1].close()

### Decode and print bounding box results

In [ ]:
text_results = {}

for part in results.parts:    
    # part is a BodyPart object with b'Content-Type', and b'Content-Disposition'; 
    # the latter includes 'name' and 'filename' info.
    headers = {}
    for k, v in part.headers.items():
        headers[k.decode(part.encoding)] = v.decode(part.encoding)
    if headers.get('Content-Type', None) == 'application/json':
        assert len(text_results) == 0
        text_results = json.loads(part.content.decode())

for image_name in text_results.keys():
    print('Bounding boxes for {}:'.format(image_name))
    print(text_results[image_name])
    print('')

### Decode and display rendered images

In [ ]:
images = {}
for part in results.parts:
    # part is a BodyPart object with b'Content-Type', and b'Content-Disposition';
    # the latter includes 'name' and 'filename' info
    headers = {}
    for k, v in part.headers.items():
        headers[k.decode(part.encoding)] = v.decode(part.encoding)
    if headers.get('Content-Type', None) == 'image/jpeg':
        c = headers.get('Content-Disposition')
        image_name = c.split('name="')[1].split('"')[0]  # somehow all the filename and name info is all in one string with no obvious forma
        image = Image.open(io.BytesIO(part.content))        
        images[image_name] = image
    
    elif headers.get('Content-Type', None) == 'application/json':
        text_result = json.loads(part.content.decode())

for img_name, img in sorted(images.items()):
    print(img_name)
    display(img)
    print('')